<a href="https://colab.research.google.com/github/Alan-Cheong/IEEE_QW_2020/blob/master/QAOA_Qiskit_Takashimaya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Using QAOA for Takashimaya maxcut layout
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit-ibm-provider
!pip install qiskit_algorithms

from qiskit_aer import Aer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# Define a simplified version of the mall layout
# Let's assume we have 5 stores (A, B, C, D, E) we want to place in a 2x3 grid
# 0 represents an empty space, 1 represents a store
# We aim to place stores in a way that maximizes adjacency (flow)

# Example objective: Maximize the number of adjacent stores
# This is a very basic model, in reality, pedestrian flow would be much more complex

def create_mall_layout_problem():
    # Create a Quadratic Program
    qp = QuadraticProgram()

    # Define binary variables for each cell in the grid (2x3)
    for i in range(2):
        for j in range(3):
            qp.binary_var(f'x_{i}_{j}')

    # Objective function: Encourage adjacency
    # For simplicity, we'll just count adjacent pairs
    # (x00, x01), (x01, x02), (x10, x11), (x11, x12), (x00, x10), (x01, x11), (x02, x12)
    adjacency_pairs = [('x_0_0', 'x_0_1'), ('x_0_1', 'x_0_2'), ('x_1_0', 'x_1_1'), ('x_1_1', 'x_1_2'),
                       ('x_0_0', 'x_1_0'), ('x_0_1', 'x_1_1'), ('x_0_2', 'x_1_2')]

    for pair in adjacency_pairs:
        qp.maximize(linear={pair[0]: 1, pair[1]: 1}, quadratic={(pair[0], pair[1]): 1})

    # Constraints: Place exactly 5 stores
    qp.linear_constraint(linear={f'x_{i}_{j}': 1 for i in range(2) for j in range(3)}, sense='==', rhs=5, name='total_stores')

    return qp

# Solve the problem using QAOA
def solve_with_qaoa(qp):
    # Convert to QUBO
    qubitOp, offset = qp.to_ising()

    # Define the backend
    backend = Aer.get_backend('qasm_simulator')

    # Define the optimizer
    optimizer = COBYLA()

    # Define the QAOA instance
    qaoa = QAOA(optimizer, quantum_instance=backend, p=2, reps=2)

    # Solve the problem
    meo = MinimumEigenOptimizer(qaoa)
    result = meo.solve(qp)

    return result

# Run the optimization
mall_layout_problem = create_mall_layout_problem()
result = solve_with_qaoa(mall_layout_problem)

# Print the result
print("Optimal store placement:", result.optimal_solution)
print("Optimal value:", result.optimal_value)

# Note: The output will be in binary form, where '1' indicates a store and '0' an empty space.
# You would need to map this back to your mall layout for visualization or further analysis.

ModuleNotFoundError: No module named 'qiskit.algorithms'